# End2end approach

> Autoencoder + forecaster in the same training loop

In [5]:
import sys
sys.path.append('..')
from mocatml.utils import *
convert_uuids_to_indices()
from mocatml.data import *
import h5py
from tsai.basics import *
my_setup()
from fastai.callback.schedule import valley, steep
from fastai.callback.wandb import WandbCallback
import wandb

os              : Linux-6.2.0-36-generic-x86_64-with-glibc2.31
python          : 3.9.17
tsai            : 0.3.8
fastai          : 2.7.13
fastcore        : 1.5.29
torch           : 2.1.1+cu121
device          : 1 gpu (['NVIDIA GeForce RTX 3050 Ti Laptop GPU'])
cpu cores       : 14
threads per cpu : 1
RAM             : 15.3 GB
GPU memory      : [4.0] GB


In [6]:
config_base = yaml2dict('./config/base.yaml', attrdict=True)
config_e2e = yaml2dict('./config/e2e.yaml', attrdict=True)
config = AttrDict({**config_base, **config_e2e})
config

```json
{ 'bs': 64,
  'data': {'path': '~/data/mc_density_data/comb_am_rp.npy'},
  'horizon': 1,
  'lookback': 12,
  'lr_max': None,
  'mmap': 'True,',
  'n_epoch': 25,
  'seed': 42,
  'sel_steps': None,
  'stride': 6,
  'tmp_folder': 'tmp',
  'wandb': { 'enabled': False,
             'group': None,
             'log_learner': False,
             'mode': 'offline',
             'project': 'mocatml'}}
```

In [7]:
run = wandb.init(project=config.wandb.project, 
                 config=config,
                 group=config.wandb.group,
                 mode=config.wandb.mode, 
                 anonymous='never') if config.wandb.enabled else None
config = dict2attrdict(run.config) if config.wandb.enabled else config
print(config)

{'bs': 64, 'data': {'path': '~/data/mc_density_data/comb_am_rp.npy'}, 'lr_max': None, 'mmap': 'True,', 'n_epoch': 25, 'seed': 42, 'tmp_folder': 'tmp', 'wandb': {'enabled': False, 'log_learner': False, 'mode': 'offline', 'group': None, 'project': 'mocatml'}, 'lookback': 12, 'horizon': 1, 'sel_steps': None, 'stride': 6}


In [8]:
data = np.load(Path(config.data.path).expanduser(), 
            mmap_mode='c' if config.mmap else None)

In [10]:
np_save_compressed(data[:10, :100], 'TLE_density_10_15x15.npy', path='../example_data')

In [17]:
splits = RandomSplitter()(data)
splits

((#80) [40,4,92,86,41,7,79,46,28,50...],
 (#20) [11,19,9,27,84,99,65,0,10,53...])

In [18]:
X_train, y_train = apply_sliding_window_3d(data[splits[0]], 
                                           window_len=config.lookback, 
                                           horizon=config.horizon, 
                                           stride=config.stride, 
                                           seq_first=False)

In [ ]:
apply_sliding_window_3d??

Signature:
apply_sliding_window_3d(
    data,
    window_len,
    *,
    stride: 'Union[None, int]' = 1,
    start: 'int' = 0,
    pad_remainder: 'bool' = False,
    padding: 'str' = 'post',
    padding_value: 'float' = nan,
    add_padding_feature: 'bool' = True,
    get_x: 'Union[None, int, list]' = None,
    get_y: 'Union[None, int, list]' = None,
    y_func: 'Optional[callable]' = None,
    output_processor: 'Optional[callable]' = None,
    copy: 'bool' = False,
    horizon: 'Union[int, list]' = 1,
    seq_first: 'bool' = True,
    sort_by: 'Optional[list]' = None,
    ascending: 'bool' = True,
    check_leakage: 'bool' = True,
)
Source:   
@delegates(SlidingWindow)
def apply_sliding_window_3d(data, window_len, **kwargs):
    """
        Apply sliding window to 3D data. The data is assumed to have the shape
        (n_samples, n_features, n_time_steps).
        Input:
            data: 3D array
            window_len: int, length of the sliding window
            horizon: int, numb